In [119]:
import os, sys, pickle

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

import jp_mpl as jplot
import random
from copy import deepcopy

# My tools
import jp_analysis as analysis
reload(analysis)

from scipy import optimize

In [421]:
import reflection_fit

In [779]:
pmt_info = pickle.load(open('/home/jpyanez/snoplus/snoplus_python/pmt_positions.pckl'))
shadowed= pickle.load(open('/home/jpyanez/scratch/SOC_shadows/SOC_0000102574.pckl'))
pmtbool = (pmt_info['type'] == 1)+(pmt_info['type'] == 7) * ~shadowed
c = 0.299792458*1000 # mm/ns, same as in Geant4

In [780]:
# Settings
#myinput = '/home/jpyanez/scratch/laserball/337/p300_n300_0'
#myinput = '/home/jpyanez/scratch/laserball/337/p300_n300_p400'


#myinput = '/home/jpyanez/scratch/laserball/337/0_n300_p400'

myinput = '/home/jpyanez/scratch/laserball/337/0_0_n200'

#myinput = '/home/jpyanez/scratch/laserball/337/0_n300_n300'

#myinput = '/home/jpyanez/scratch/laserball/337/p300_0_0'

#myinput = '/home/jpyanez/scratch/laserball/337/0_0_0'
#myinput = '/home/jpyanez/scratch/laserball/505/0_0_0'

analyze_fraction = 1.

In [781]:
# Make it work either for a directory or a file
if os.path.isdir(myinput):
    infile_list = [os.path.join(myinput,x) for x in os.listdir(myinput) if '.pckl' in x]
else:
    infile_list = [myinput]    

In [782]:
# Randomize the list of files
random.shuffle(infile_list)

In [783]:
# Select a subset of files
total_files = int(np.ceil(len(infile_list)*analyze_fraction))
infile_list = infile_list[:total_files]

In [784]:
# Open infiles, load and sum the histograms
sumkeys = ['toa_map', 'occupancy']
for iFile, fname in enumerate(infile_list):
    fopen     = open(fname)
    this_data = pickle.load(fopen)
    if iFile == 0:
        data = deepcopy(this_data)
    else:
        for one_key in sumkeys:
            if this_data.has_key(one_key):
                data[one_key] += this_data[one_key]
            else:
                print fname, ' has no ', one_key
    fopen.close()
    

/home/jpyanez/scratch/laserball/337/0_0_n200/8_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/12_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/6_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/4_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/15_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/2_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/9_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/18_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/13_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/10_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/14_lb_wn_337_TOA.pckl  has no  occupancy
/home/jpyanez/scratch/laserball/337/0_0_n200/19_lb_wn_337_T

In [785]:
print 'Maximum PMT occupancy ', data['toa_map'].max()

Maximum PMT occupancy  460000.0


print 'PMT time of arrival map'
plt.imshow(np.log10(data['toa_map']).T, origin='lower', aspect='auto')
plt.colorbar()

In [786]:
data.keys()

['time_edges', 'toa_map', 'manip_pos', 'occupancy']

In [787]:
toa_peak = np.zeros([pmt_info['type'].size, 4])
toa_dpeak = np.zeros_like(toa_peak)
toa_gaus = np.zeros_like(toa_peak)

In [788]:
reload(reflection_fit)

<module 'reflection_fit' from '/home/jpyanez/snoplus/snoplus_python/reflection_fit.pyc'>

In [789]:
for iPMT in range(toa.shape[0]):
    toa_peak[iPMT,:] = reflection_fit.getPeakTimes( n = data['toa_map'][iPMT,:],
                                               ybins = data['time_edges'],
                                                  refl_peak_width=4.5)
    toa_dpeak[iPMT,:] = reflection_fit.getDPeakTimes( n = data['toa_map'][iPMT,:],
                                               ybins = data['time_edges'],
                                                  refl_peak_width=4.5 )
    toa_gaus[iPMT,:] = reflection_fit.getGausTimes( n = data['toa_map'][iPMT,:],
                                               ybins = data['time_edges'],
                                                  refl_peak_width=4.5 )

In [790]:
# Defining GOOD data
peakbool= (toa_peak[:,0] > 0) * (toa_peak[:,2]>0)
dpeakbool =  (toa_dpeak[:,0] > 0) * (toa_dpeak[:,2]>0)
gausbool = (toa_gaus[:,0]>0)*(toa_gaus[:,2]>0)

In [791]:
# Defining the tdiff and errors
peak_tdiff  = toa_peak[:,2] - toa_peak[:,0]
dpeak_tdiff = toa_dpeak[:,2] - toa_dpeak[:,0]
gaus_tdiff  = toa_gaus[:,2] - toa_gaus[:,0]

# Defining errors
peak_err  = dpeak_err = np.ones_like(peak_tdiff)
gaus_err  = np.sqrt(toa_gaus[:,1]**2 + toa_gaus[:,3]**2)

In [792]:
peakfit = reflection_fit.FitLBpos( data = peak_tdiff,
                                   error = peak_err,
                                   pmt_xyz = pmt_info['xyz'],
                                   pmtbool = peakbool*pmtbool,
                                   print_call = False)

In [793]:
dpeakfit = reflection_fit.FitLBpos( data = dpeak_tdiff,
                                   error = dpeak_err,
                                   pmt_xyz = pmt_info['xyz'],
                                   pmtbool = dpeakbool*pmtbool,
                                   print_call = False)

In [794]:
gausfit = reflection_fit.FitLBpos( data = gaus_tdiff,
                                   error = gaus_err,
                                   pmt_xyz = pmt_info['xyz'],
                                   pmtbool = gausbool*pmtbool,
                                   print_call = False)

In [795]:
wrapfcn = lambda p: peakfit(*p)
fit_counter = 0
repeat_fit = True
while repeat_fit:
    my_x0 = np.concatenate((data['manip_pos']*(1+np.random.rand(3)/10.) + np.random.rand(3)/20., [1.3]))
    peakr = optimize.minimize(wrapfcn,
                        x0 = my_x0,
                        method = 'SLSQP',
                        bounds=((-6500,6500),
                                (-6500,6500),
                                (-6500,6500),
                                (0.5, 2.)),
                        options={'ftol':1E-7, 'maxiter':1000})
    fit_counter += 1
    if peakr.success:
        repeat_fit = False
        print 'PEAK good fit at ', fit_counter
    if fit_counter >= 20:
        print 'PEAK Fit failed! Check why'
        repeat_fit = False

PEAK good fit at  2


In [796]:
wrapfcn = lambda p: dpeakfit(*p)
fit_counter = 0
repeat_fit = True
while repeat_fit:
    my_x0 = np.concatenate((data['manip_pos']*(1+np.random.rand(3)/10.) + np.random.rand(3)/20., [1.3]))
    dpeakr = optimize.minimize(wrapfcn,
                        x0 = my_x0,
                        method = 'SLSQP',
                        bounds=((-6500,6500),
                                (-6500,6500),
                                (-6500,6500),
                                (0.5, 2.)),
                        options={'ftol':1E-7, 'maxiter':1000})
    fit_counter += 1
    if dpeakr.success:
        repeat_fit = False
        print 'DPEAK good fit at ', fit_counter
    if fit_counter >= 20:
        print 'PEAK Fit failed! Check why'
        repeat_fit = False

PEAK Fit failed! Check why


In [797]:
wrapfcn = lambda p: gausfit(*p)
fit_counter = 0
repeat_fit = True
while repeat_fit:
    my_x0 = np.concatenate((data['manip_pos']*(1+np.random.rand(3)/10.) + np.random.rand(3)/100., [1.3]))
    gausr = optimize.minimize(wrapfcn,
                        x0 = my_x0,
                        method = 'SLSQP',
                        bounds=((-6500,6500),
                                (-6500,6500),
                                (-6500,6500),
                                (0.5, 2.)),
                        options={'ftol':1E-7, 'maxiter':1000})
    fit_counter += 1
    if gausr.success:
        repeat_fit = False
        print 'GAUS good fit at ', fit_counter
    if fit_counter >= 20:
        print 'GAUS Fit failed! Check why'
        repeat_fit = False

GAUS Fit failed! Check why


In [798]:
print 'Peaks', peakr.x
print 'Derivative', dpeakr.x
print 'Gaus fit', gausr.x

Peaks [   3.03903302   -4.1566993  -171.322532      1.40789053]
Derivative [  2.26323017e-02   1.96636049e-02  -2.12199665e+02   1.30000000e+00]
Gaus fit [ nan  nan  nan  nan]


In [799]:
print 'Peaks:', peakr.message
print 'Derivative:', dpeakr.message
print 'Gaus fit:', gausr.message

Peaks: Optimization terminated successfully.
Derivative: Positive directional derivative for linesearch
Gaus fit: Iteration limit exceeded


In [800]:
outdir = '/home/jpyanez/scratch/laserball_results'

In [801]:
results = {'manip': data['manip_pos'],
           'fit_peaks':peakr.x,
           'fit_dpeaks':dpeakr.x,
           'fit_gaus':gausr.x}

In [802]:
myinput.split('/')[-2]+ '_' + myinput.split('/')[-1]

'337_0_0_n200'

In [803]:
outfile_name = os.path.join(outdir, myinput.split('/')[-2]+ '_' + myinput.split('/')[-1] + '.pckl')

In [804]:
pickle.dump(results, open(outfile_name, 'w'))